# Data Analysis on V5 API

## Research questions:

1. What's the number of challenges by status over different granularity (e.g. year, month, week)?
  - Specifically, what does the cancelled challenges' distribution looked like?
2. There are 4 tracks now, a new **"QA"** track is added. What's the mapping from legacy track/sub-track to new track?
3. There are 3 types of challenge now, **"first to finish"** and **"task"** are separated from **"challenge"**, where do these two new challenges come from (legacy track/sub track)?
4. * Looking at the prize set of the challenges, what's the distrobution in different track/legacy track?

In [ ]:
%matplotlib inline
import os
import re
from pprint import pprint
from collections import defaultdict
from datetime import datetime

import pymongo
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.dates as mdates

import topcoder_mongo as DB
import static_var as S
import util as U

sns.set(
    rc={
        'axes.facecolor':'#121212',
        'figure.facecolor':'#121212',
        'text.color': 'white',
        'axes.titlecolor': 'white',
        'axes.labelcolor': 'white',
        'xtick.color': 'white',
        'ytick.color': 'white'
    },
)

pd.set_option('display.max_rows', 500)

## Plots and analysis

> To reduce the length of variable, abbrevations of words are used.
> Usually when you see `cha_*` it means `challenges_*`.

### Number of Challenges by time range.

In [ ]:
query = [
    {'$match': {'end_date': {'$lte': U.year_end(2020)}}},
    {
        '$project': {
            'status': {'$arrayElemAt': [{'$split': ['$status', ' - ']}, 0]},
            'end_date': '$end_date',
        },
    },
    {'$group': {'_id': {'status': '$status'}, 'count': {'$sum': 1}}},
    {'$replaceRoot': {'newRoot': {'$mergeObjects': ['$_id', {'count': '$count'}]}}},
]

cha_cnt_status_df = pd.DataFrame.from_records(
    data=DB.TopcoderMongo.run_challenge_aggregation(query),
)

fig = plt.figure(figsize=(8, 4), dpi=200)
ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])

sns.despine()
sns.barplot(
    data=cha_cnt_status_df,
    x='count',
    y='status',
    order=S.STATUS[::-1],
    linewidth=0,
    alpha=0.9,
    palette=['#8E7865', '#8579AA'] + ['grey'] * 4,
    ax=ax,
)

ax.grid(False)
ax.set_title('Challenge Count By Status')
ax.set_xlabel('Count')
ax.set_ylabel('Status')

for p in ax.patches:
    cnt = int(p.get_width())
    x = p.get_width()
    y = p.get_height() / 2 + p.get_y()
    ax.annotate(
        cnt,
        xy=(x, y),
        xytext=(x + 150 if cnt < 140 else x - 150, y),
        va='center',
        ha= 'left' if cnt < 140 else 'right',
        color='red' if cnt < 140 else 'white'
    )

In [ ]:
cha_cnt_year_df = pd.DataFrame.from_records(
    data=DB.TopcoderMongo.get_challenge_count('year', '%Y', S.STATUS[:-2]),
).set_index(['year', 'status']).sort_index()

fig, ax = plt.subplots(1, 1, figsize=(16, 4), dpi=200)
sns.despine()

data = (cha_cnt_year_df
        .groupby('year')
        .sum()\
        .reset_index()
        .apply({'year': lambda dt: dt.strftime('%Y'), 'count': lambda cnt: cnt}))

sns.barplot(
    data=data,
    x='year',
    y='count',
    linewidth=0,
    color='grey',
    alpha=0.9,
    ax=ax,
)

ax.grid(False)
ax.set_title('Number of challenges by year')
ax.set_xlabel('Year')
ax.set_ylabel('Count')

# set the width of bar to be connected to each other
for p in ax.patches:
#     p.set_width(1)
#     p.set_x(p.get_x() - 0.1)
    
    cnt = p.get_height()
    x = p.get_x() + p.get_width() / 2
    y = p.get_height()
    ax.annotate(int(cnt), xy=(x, y), xytext=(x, y + 50), ha='center')

# fig.autofmt_xdate()

In [ ]:
color_palette = list(sns.color_palette(n_colors=4).as_hex())
fig, ax = plt.subplots(1, 1, figsize=(16, 4), dpi=200)
sns.despine()

challenge_count_by_year = defaultdict(dict)
for idx, status in enumerate(S.STATUS[:-2]):
    challenge_count_by_year[status]['data'] = cha_cnt_year_df.loc[(slice(None), status), :].reset_index('status', drop=True)
    challenge_count_by_year[status]['color'] = color_palette[idx]
    if idx == 0:
        challenge_count_by_year[status]['bottom'] = pd.DataFrame({'count': [0] * 7}, index=cha_cnt_year_df.index.get_level_values('year').unique())
        
    else:
        challenge_count_by_year[status]['bottom'] = cha_cnt_year_df.loc[(slice(None), S.STATUS[:idx]), :].groupby('year').sum()

for status, data_dct in challenge_count_by_year.items():
    sns.barplot(
        x=data_dct['data'].index.strftime('%Y'),
        y=data_dct['data']['count'],
        bottom=data_dct['bottom']['count'],
        color=data_dct['color'],
        linewidth=0,
        label=status,
        ax=ax,
    )
    
# ax.set_xticklabels(cha_cnt_year_df.index.get_level_values('year').unique().strftime('%Y'))
ax.grid(False)
ax.set_title('Number of challenges by year')
ax.set_xlabel('Year')
ax.set_ylabel('Count')
ax.legend()

text_coords = defaultdict(int)
for p in ax.patches:
    x = p.get_x() + p.get_width() / 2
    y = p.get_height()
    text_coords[x] += y
    
for x, y in text_coords.items():
    cnt = int(y)
    ax.annotate(cnt, xy=(x, y), xytext=(x, y + 50), ha='center')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 4), dpi=200)
sns.despine()

status = 'Cancelled'
status_data = challenge_count_by_year[status]

sns.barplot(
    data=data,
    x='year',
    y='count',
    color='grey',
    linewidth=0,
    alpha=0.9,
    ax=ax,
)

sns.barplot(
    x=status_data['data'].index.strftime('%Y'),
    y=status_data['data']['count'],
    color=status_data['color'],
    linewidth=0,
    label=status,
    ax=ax,
)

ax.grid(False)
ax.set_title('Number of challenges by year')
ax.set_xlabel('Year')
ax.set_ylabel('Count')
ax.legend()

text_coords = defaultdict(list)
for p in ax.patches:
    x = p.get_x() + p.get_width() / 2
    y = p.get_height()
    text_coords[x].append(y)
    
for x, ys in text_coords.items():
    y_part, y_all = sorted(ys)
    ax.annotate(int(y_part), xy=(x, y_part), xytext=(x, y_part + 50), ha='center')
    ax.annotate(int(y_all), xy=(x, y_all), xytext=(x, y_all + 50), ha='center')


In [ ]:
cha_cnt_month_df = pd.DataFrame.from_records(
    data=DB.TopcoderMongo.get_challenge_count('month', '%Y-%m', S.STATUS[:-2]),
).set_index(['month', 'status']).sort_index()

data = cha_cnt_month_df.groupby('month').sum()

fig, ax = plt.subplots(1, 1, figsize=(16, 4), dpi=200)
sns.despine()

# abnormal_palette = data['count'].apply(lambda cnt: 'grey' if cnt < 800 else 'red').tolist()

sns.barplot(
    x=data.index,
    y=data['count'],
    linewidth=0.1,
    color='grey',
    alpha=0.9,
#     palette=abnormal_palette,
    ax=ax,
)

index_tup = [dt_str.split() for dt_str in data.index.strftime('%Y %b')]
xticklabels = []
for year, month in index_tup:
    if month == 'Jan':
        xticklabels.append(f'{month}\n{year}')
    elif month in ['Apr', 'Jul', 'Oct']:
        xticklabels.append(month)
    else:
        xticklabels.append(' ')
        
ax.set_xticklabels(xticklabels)
ax.set_yticks(list(range(0, 1501, 300)))
ax.set_yticklabels(list(range(0, 1501, 300)))

ax.set_title('Number of Challenges by Month')
ax.set_xlabel('Month')
ax.set_ylabel('Count')

for p in ax.patches:
    p.set_width(1)
    p.set_x(p.get_x() - 0.1)

In [ ]:
challenge_count_by_month = defaultdict(dict)
for idx, status in enumerate(S.STATUS[:-2]):
    challenge_count_by_month[status]['data'] = cha_cnt_month_df.loc[(slice(None), status), :].reset_index('status', drop=True)
    challenge_count_by_month[status]['color'] = color_palette[idx]
    if idx == 0:
        challenge_count_by_month[status]['bottom'] = pd.DataFrame({'count': 0}, index=cha_cnt_month_df.index.get_level_values('month').unique())
        
    else:
        challenge_count_by_month[status]['bottom'] = cha_cnt_month_df.loc[(slice(None), S.STATUS[:idx]), :].groupby('month').sum()

fig, ax = plt.subplots(1, 1, figsize=(16, 4), dpi=200)
sns.despine()
        
for status, data_dct in challenge_count_by_month.items():
    sns.barplot(
        x=data_dct['data'].index.strftime('%Y-%m'),
        y=data_dct['data']['count'],
        bottom=data_dct['bottom']['count'],
        color=data_dct['color'],
        linewidth=0.1,
        label=status,
        ax=ax,
    )
    
ax.set_xticklabels(xticklabels)
ax.set_yticks(list(range(0, 1501, 300)))
ax.set_yticklabels(list(range(0, 1501, 300)))
ax.legend()

ax.set_title('Number of Challenges by Month')
ax.set_xlabel('Month')
ax.set_ylabel('Count')

for p in ax.patches:
    p.set_width(1)
    p.set_x(p.get_x() - 0.1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 4), dpi=200)
sns.despine()

status = 'Completed'
status_data = challenge_count_by_month[status]

sns.barplot(
    x=data.index,
    y=data['count'],
    color='grey',
    linewidth=0.1,
    alpha=0.9,
    label='Total',
    ax=ax,
)

sns.barplot(
    x=status_data['data'].index,
    y=status_data['data']['count'],
    color=status_data['color'],
    linewidth=0.1,
    label=status,
    ax=ax,
)

ax.set_xticklabels(xticklabels)
ax.set_yticks(list(range(0, 1501, 300)))
ax.set_yticklabels(list(range(0, 1501, 300)))
ax.legend()

ax.set_title('Number of Challenges by Month')
ax.set_xlabel('Month')
ax.set_ylabel('Count')

# ax.set_ylim(0, 300)
# ax.set_yticks(list(range(0, 301, 50)))
# ax.set_yticklabels(list(range(0, 301, 50)))

# ax.set_ylim(0, 600)
# ax.set_yticks(list(range(0, 601, 150)))
# ax.set_yticklabels(list(range(0, 601, 150)))

for p in ax.patches:
    p.set_width(1)
    p.set_x(p.get_x() - 0.1)


In [ ]:
proj_ov_df = pd.DataFrame.from_records(
    DB.TopcoderMongo.run_project_aggregation([
        {'$match': {'id': {'$ne': None}}},
        {
            '$project': {
                'id': True,
                'start_date': True,
                'end_date': True,
                'num_of_challenge': '$num_of_challenge.Total'
            }
        },
        {'$match': {'num_of_challenge': {'$gte': 100}}},
        {'$project': {'_id': False}},
        {'$sort': {'num_of_challenge': -1}}
    ])
)

fig, ax = plt.subplots(1, 1, figsize=(16, 4), dpi=200)
sns.despine()

ax.xaxis_date()
ax.set_xlim(U.year_start(2014), U.year_end(2020))
ax.set_yticks(list(range(0, 1000, 200)))

# Configure the xaxis ticks
year_lctr = mdates.YearLocator()
year_fmt = mdates.DateFormatter('\n%Y')
month_lctr = mdates.MonthLocator(bymonth=[1, 4, 7, 10])
month_fmt = mdates.DateFormatter('%b')
ax.xaxis.set_major_locator(year_lctr)
ax.xaxis.set_major_formatter(year_fmt)
ax.xaxis.set_minor_locator(month_lctr)
ax.xaxis.set_minor_formatter(month_fmt)
ax.xaxis.set_remove_overlapping_locs(False)
ax.xaxis.grid(False)

ax.set_title('Project Span vs. Number of Challenges')
ax.set_xlabel('Month')
ax.set_ylabel('Number of Challenge')

for project in proj_ov_df.itertuples(index=False):
    sns.lineplot(
        x=[project.start_date, project.end_date],
        y=[project.num_of_challenge, project.num_of_challenge],
        linewidth=15,
        alpha=0.5,
        color='#8172B3',
        ax=ax,
    )
    
    x = project.start_date + (project.end_date - project.start_date) / 2
    y = project.num_of_challenge
    ax.annotate(project.num_of_challenge, xy=(x, y), xytext=(x, y), ha='center', va='center')


### Comparison between old track/sub-track to new track and challenge type

In [ ]:
query = [
    {'$match': {'legacy.track': {'$exists': True}, 'legacy.sub_track': {'$exists': True}}},
    {
        '$group': {
            '_id': {
                'track': '$track',
                'type': '$type',
                'legacy_track': '$legacy.track',
                'legacy_sub_track': '$legacy.sub_track'
            },
            'count': {'$sum': 1},
        },
    },
    {'$replaceRoot': {'newRoot': {'$mergeObjects': ['$_id', {'count': '$count'}]}}},
]

track_type_df = pd.DataFrame.from_records(
    DB.TopcoderMongo.run_challenge_aggregation(query)
).set_index(['track', 'type', 'legacy_track', 'legacy_sub_track']).sort_index()

In [ ]:
track_type_df.groupby(['legacy_track', 'track']).sum()

In [ ]:
(track_type_df
 .loc[track_type_df.index.get_level_values('type').isin(['First2Finish', 'Task']), :]
 .groupby(['type', 'legacy_sub_track'])
 .sum())

In [ ]:
(track_type_df
 .loc[track_type_df.index.get_level_values('legacy_sub_track').isin(['FIRST_2_FINISH', 'DESIGN_FIRST_2_FINISH']), :]
 .groupby(['legacy_sub_track', 'type'])
 .sum())

In [ ]:
(track_type_df
 .loc[track_type_df.index.get_level_values('track') == 'Quality Assurance', :]
 .groupby(['track', 'type', 'legacy_sub_track'])
 .sum())

In [ ]:
groupby_legacy_track = track_type_df.groupby('legacy_track').sum().sort_index(ascending=False).reset_index()
groupby_track = (track_type_df.groupby('track')
                 .sum()
                 .reindex([
                     'Development',
                     'Quality Assurance',
                     'Design',
                     'Data Science'
                 ])
                 .reset_index()
                )
groupby_both = track_type_df.groupby(['track', 'legacy_track']).sum()

In [ ]:
fig, (ax0, ax1) = plt.subplots(2, 1, figsize=(16, 4), dpi=200, sharex=True)

legacy_track_color = {
    'DEVELOP': '#586DB4',
    'DESIGN': '#DD60A7',
    'DATA_SCIENCE': '#FF7D4E',
}
track_color = ['#006999', '#9F69BA', '#FF6280', '#FFA600']

# ------------------ Plot legacy track distro ------------------

ax0.set_title('Legacy Track Distribution')
for row in groupby_legacy_track.itertuples():
    left = 0 if row.Index == 0 else groupby_legacy_track.loc[groupby_legacy_track.index < row.Index, 'count'].sum()
    sns.barplot(
        x=[row.count],
        y=[''],
        label=row.legacy_track,
        color=legacy_track_color[row.legacy_track],
        left=left,
        linewidth=0,
        ax=ax0,
    )
    
for p in ax0.patches:
    p.set_height(0.5)
    p.set_y(-0.4)
    
    cnt = int(p.get_width())
    x = p.get_x()
    y = p.get_y() + p.get_height() / 2
    ax0.annotate(int(cnt), xy=(x, y), xytext=(x + 150, y), color='white', ha='left', va='center')
    
# ------------------ Plot track distro ------------------

ax1.set_title('Track Distribution')
for row in groupby_track.itertuples():
    left = 0 if row.Index == 0 else groupby_track.loc[groupby_track.index < row.Index, 'count'].sum()
    sns.barplot(
        x=[row.count],
        y=[''],
        label=row.track,
        color=track_color[row.Index],
        left=left,
        linewidth=0,
        ax=ax1,
    )

for p in ax1.patches:
    p.set_height(0.5)
    p.set_y(-0.4)
    
    cnt = int(p.get_width())
    x = p.get_x()
    y = p.get_y() + p.get_height() / 2
    ax1.annotate(int(cnt), xy=(x, y), xytext=(x + 150, y), color='white', ha='left', va='center')
    
for ax in (ax0, ax1):
    sns.despine(ax=ax, left=True, bottom=True)
    ax.grid(False)
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles, labels, ncol=len(handles), loc=8)

    ax.set_xticks([])
    ax.set_xlim(0, track_type_df['count'].sum())
    
fig.tight_layout()

### Explore the relationship between tracks and prize set